In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import glob

#表示行数の増加
pd.set_option('display.max_rows', 500)

today = datetime.datetime.now()

y = today.year
m = today.month - 1

ym = str(y) + str('{0:02d}'.format(m))

#暫定でのroot dir
root = "/Users/miyamoto/Documents/Root/raw"

In [2]:
def file_ditect(key_dir):#指定ディレクトリに存在するファイルパスの一覧取得
    path = root + "/" + key_dir + "/"
    
    files = []

    # パスで指定したファイルの一覧をリスト形式で取得
    files = glob.glob(path + '/*.csv')

    #print(files)

    for file in files:
        print(file)
    
    return files
        
def aggregate(df):
    df_agg = \
    df.groupby(["属性", "ショップ名", "ブランド名"])\
    .agg({'受注可能在庫数':'sum', '受注数':'sum','受注金額':'sum', 'SKU数':'sum', '欠損SKU数':'sum'})
    
    df_agg["欠損率"] = df_agg["欠損SKU数"] /  df_agg["SKU数"]
    df_agg["回転数"] = df_agg["受注可能在庫数"] /  df_agg["受注数"]
    
    return df_agg
    

In [3]:
def import_csv(df, file):
    df = df[~df["ブランド名"].str.contains("ノベルティ")]
    df["属性"] = "受注金額"
    df = df.groupby(["属性", "ショップ名", "ブランド名"]).agg({"受注金額":"sum"})
    
    return df

In [4]:
files = file_ditect("missingRate")
order_files = file_ditect("missingRate/orderSum")

/Users/miyamoto/Documents/Root/raw/missingRate/☆欠損率_CAN_20210201095351.csv
/Users/miyamoto/Documents/Root/raw/missingRate/☆欠損率_レポート（新）_20210201095446.csv
/Users/miyamoto/Documents/Root/raw/missingRate/☆欠損率_STRIPE INTERNATIONAL（新）_20210201095412.csv
/Users/miyamoto/Documents/Root/raw/missingRate/☆欠損率_レポート_00_20210201095429.csv
/Users/miyamoto/Documents/Root/raw/missingRate/orderSum/受注_20210201095907_AP.csv
/Users/miyamoto/Documents/Root/raw/missingRate/orderSum/受注_20210201095536_new.csv
/Users/miyamoto/Documents/Root/raw/missingRate/orderSum/受注_20210201095747_zero.csv
/Users/miyamoto/Documents/Root/raw/missingRate/orderSum/ランキング_20210201095955_AP_rank.csv
/Users/miyamoto/Documents/Root/raw/missingRate/orderSum/受注_20210201095835_CAN.csv
/Users/miyamoto/Documents/Root/raw/missingRate/orderSum/受注_20210201095615_stripe.csv


In [5]:
df_list = []
for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

df_all = pd.concat(df_list, axis=0)

df_list_order = []

for file in order_files:
    if "AP_rank" in file:
        df_ap =  pd.read_csv(file)
        df_ap["属性"] = "ランキング"
        df_ap = df_ap[~df_ap["ブランド名"].str.contains("ノベルティ")]
        df_ap = df_ap.query("ショップ名 == 'Arpege story' & 商品コード != '[その他]' ")
        df_ap["欠損SKU数"] = np.where(df_ap["受注可能在庫数"]>0 , 0, 1)
    else:
        df = pd.read_csv(file)
        df = import_csv(df, file)
        df_list_order.append(df)

df_order_all = pd.concat(df_list_order, axis=0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_orderSum = df_all.groupby(["ショップ名", "ブランド名", "商品コード"]).agg({"受注金額":"sum"})
df_orderSum.rename(columns = {"受注金額":"ランク"}, inplace = True)
df_rank = df_orderSum.groupby(["ショップ名", "ブランド名"]).rank(method="min", ascending=False)

In [7]:
df_all = pd.merge(df_all, df_rank, on = ["ショップ名", "ブランド名", "商品コード"])
df_all["受注可能在庫数"] = df_all["受注可能在庫数"].where(df_all["受注可能在庫数"] != '< NULL >',0).astype("int64")

df_all_ap = \
df_ap.groupby(["属性", "ショップ名", "ブランド名"])\
.agg({"SKUコード":"nunique","受注数":"sum","欠損SKU数":"sum","受注可能在庫数":"sum", "受注金額":"sum"})
df_all_ap.rename(columns = {"SKUコード":"SKU数"}, inplace = True)

In [8]:
df_Top10 = df_all.query("ランク <= 10")
df_Top10["属性"] = "金額ベスト10"

df_Top30 = df_all.copy()
df_Top30["属性"] = "金額ベスト30"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df_Top10_agg = aggregate(df_Top10)
df_Top30_agg = aggregate(df_Top30)

In [10]:
df_all_ap

SKU数  受注数  欠損SKU数  受注可能在庫数     受注金額
属性    ショップ名        ブランド名                                               
ランキング Arpege story Apuweiser-riche   143  600       7     1493  9511660
                   Arpege story       30   47      10       92   457880
                   CADUNE             79  154      23      195  2960600
                   JUSGLITTY          97  294       6      930  5000900
                   Mystrada          111  278      10      909  4685100
                   Rirandture         90  189      12      617  2867650

In [11]:
df_order_all.query("ショップ名 == 'Arpege story'")

受注金額
属性   ショップ名        ブランド名                    
受注金額 Arpege story Apuweiser-riche  13247890
                  Arpege story       457880
                  CADUNE            2960600
                  JUSGLITTY         7614510
                  Mystrada          5558990
                  Rirandture        3604400

In [12]:
df_Top10_agg

受注可能在庫数  \
属性      ショップ名                            ブランド名                                       
金額ベスト10 AURALEE                          AURALEE                                92   
                                         ノベルティ                                 210   
        Ailand                           Ank Rouge                              73   
                                         Jamie エーエヌケー                           55   
        Brown&Street ONLINE              Brown&Street                          553   
        CAN ONLINE SHOP                  Et grenier by Samansa Mos2            247   
                                         Lugnoncure                            593   
                                         Melan Cleuge men                      194   
                                         Melan Cleuge women                    812   
                                         Minun.                                 78   
                                         Samansa Mos2                         1345   
                                         Samansa Mos2 Lagom                    979   
                                         Samansa Mos2 blue                    1452   
                                         TSUHARU by Samansa Mos2               241   
                                         Te chichi                             784   
                                         Te chichi CLASSIC                      30   
                                         Te chichi TERRASSE                    656   
                                         ehka sopo                            2146   
                                         sm2rhythm                              46   
                                         sō4ū                                22595   
        CAN OUTLET                       Lugnoncure                             43   
                                         Melan Cleuge men                       77   
                                         Melan Cleuge women                    503   
                                         Samansa Mos2                           32   
                                         Samansa Mos2 Lagom                      0   
                                         Samansa Mos2 blue                      50   
                                         Te chichi                             102   
                                         Te chichi CLASSIC                       3   
                                         Te chichi TERRASSE                    113   
                                         ehka sopo                              58   
        COOMB official online store      COOMB                                  40   
        CUBE SUGAR                       CUBE SUGAR                             71   
                                         SELECT                                107   
                                         TOBILA                                  2   
        Cuore ONLINE SHOP                Flowin                                505   
        DURAS OFFICIAL WEB STORE         DURAS                                 118   
        ECLIN                            ECLIN                                  14   
        EGOIST(エゴイスト)ファッション公式通販サイト       EGOIST                                331   
        EGOIST客注専用                       EGOIST                                413   
        ENVYM                            ENVYM                                  54   
        KOE                              koe                                   113   
        LEBECCA boutique                 LEBECCA boutique                      493   
                                         ノベルティー                                 60   
        PALEMOBA ONLINESTORE             Ludic Park                           1684   
                                         Re-J＆SUPURE                           260   
        Port Of Call Online      

In [13]:
df_Top30_agg

受注可能在庫数  \
属性      ショップ名                            ブランド名                                       
金額ベスト30 AURALEE                          AURALEE                               226   
                                         ノベルティ                                 210   
        Ailand                           Ank Rouge                             306   
                                         Jamie エーエヌケー                          126   
        Brown&Street ONLINE              Brown&Street                          575   
        CAN ONLINE SHOP                  Et grenier by Samansa Mos2            488   
                                         Lugnoncure                           1672   
                                         Melan Cleuge men                      485   
                                         Melan Cleuge women                   2906   
                                         Minun.                                 78   
                                         Samansa Mos2                         7071   
                                         Samansa Mos2 Lagom                   1654   
                                         Samansa Mos2 blue                    5035   
                                         TSUHARU by Samansa Mos2               264   
                                         Te chichi                            1915   
                                         Te chichi CLASSIC                      30   
                                         Te chichi TERRASSE                   1819   
                                         ehka sopo                            4094   
                                         sm2rhythm                              46   
                                         sō4ū                                22595   
        CAN OUTLET                       Lugnoncure                            153   
                                         Melan Cleuge men                       77   
                                         Melan Cleuge women                    914   
                                         Samansa Mos2                          198   
                                         Samansa Mos2 Lagom                      0   
                                         Samansa Mos2 blue                     107   
                                         Te chichi                             227   
                                         Te chichi CLASSIC                       3   
                                         Te chichi TERRASSE                    255   
                                         ehka sopo                             127   
        COOMB official online store      COOMB                                  86   
        CUBE SUGAR                       CUBE SUGAR                            201   
                                         SELECT                                107   
                                         TOBILA                                  2   
        Cuore ONLINE SHOP                Flowin                                505   
        DURAS OFFICIAL WEB STORE         DURAS                                 370   
        ECLIN                            ECLIN                                  32   
        EGOIST(エゴイスト)ファッション公式通販サイト       EGOIST                               1033   
        EGOIST客注専用                       EGOIST                                413   
        ENVYM                            ENVYM                                 129   
        KOE                              koe                                   208   
        LEBECCA boutique                 LEBECCA boutique                      709   
                                         ノベルティー                                 60   
        PALEMOBA ONLINESTORE             Ludic Park                           1949   
                                         Re-J＆SUPURE                           710   
        Port Of Call Online      

In [14]:
df_concat = pd.concat([df_Top10_agg, df_Top30_agg, df_order_all, df_all_ap], axis = 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [15]:
df_concat.query("ショップ名 == 'Arpege story'")

SKU数  受注可能在庫数    受注数      受注金額  回転数  \
属性    ショップ名        ブランド名                                                   
受注金額  Arpege story Apuweiser-riche    NaN      NaN    NaN  13247890  NaN   
                   Arpege story       NaN      NaN    NaN    457880  NaN   
                   CADUNE             NaN      NaN    NaN   2960600  NaN   
                   JUSGLITTY          NaN      NaN    NaN   7614510  NaN   
                   Mystrada           NaN      NaN    NaN   5558990  NaN   
                   Rirandture         NaN      NaN    NaN   3604400  NaN   
ランキング Arpege story Apuweiser-riche  143.0   1493.0  600.0   9511660  NaN   
                   Arpege story      30.0     92.0   47.0    457880  NaN   
                   CADUNE            79.0    195.0  154.0   2960600  NaN   
                   JUSGLITTY         97.0    930.0  294.0   5000900  NaN   
                   Mystrada         111.0    909.0  278.0   4685100  NaN   
                   Rirandture        90.0    617.0  189.0   2867650  NaN   

                                    欠損SKU数  欠損率  
属性    ショップ名        ブランド名                         
受注金額  Arpege story Apuweiser-riche     NaN  NaN  
                   Arpege story        NaN  NaN  
                   CADUNE              NaN  NaN  
                   JUSGLITTY           NaN  NaN  
                   Mystrada            NaN  NaN  
                   Rirandture          NaN  NaN  
ランキング Arpege story Apuweiser-riche     7.0  NaN  
                   Arpege story       10.0  NaN  
                   CADUNE             23.0  NaN  
                   JUSGLITTY           6.0  NaN  
                   Mystrada           10.0  NaN  
                   Rirandture         12.0  NaN

In [16]:
df_concat.to_csv('欠損率.csv', encoding='utf_8_sig')